In [1]:
import cv2 as cv
import numpy as np

In [3]:
path = "/home/pks/Downloads/Assignment/IVP/mini project/"

In [5]:
image = cv.imread(path+"sample1.jpg", 0)

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math


def cluster(set_of_points, no_of_clusters):
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 20, 1.0)
    flags = cv.KMEANS_PP_CENTERS
    compactness,labels,centers = cv.kmeans(set_of_points,no_of_clusters,None,criteria,50,flags)
    
    labels = labels.ravel()
        
    return labels

def my_cluster(set_of_points, dist):
    temp, temp_lines = [], []
    set_of_points = sorted(set_of_points, key=lambda x: (abs(x[0]), x[1]))
    
    temp.append(set_of_points[0])
    for index,point in enumerate(set_of_points):
        if abs(abs(point[0])-abs(temp[-1][0])) <= dist:
            temp.append(point)
            
            if index == len(set_of_points)-1:
                temp_lines.append(temp[len(temp)//2])
            
        else:
            temp_lines.append(temp[len(temp)//2])
            temp = [point]
            
            if index == len(set_of_points)-1:
                temp_lines.append(point)
                
    return temp_lines


def intersection_bw_2_lines(l1, l2):
    rho1, theta1 = l1
    rho2, theta2 = l2
    
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    B = np.array([[rho1], [rho2]])
    
    x0, y0 = np.linalg.solve(A, B)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    
    return [x0, y0]


if __name__ == '__main__':
    path = "/home/pks/Downloads/Assignment/IVP/mini project/"
    image = cv.imread(path+"sample1.jpg", 0)

    cv.namedWindow('org',cv.WINDOW_NORMAL)
    cv.resizeWindow('org', 600,600)
    cv.imshow("org", image)
    cv.waitKey(0)
    
    ret, thresh = cv.threshold(image, 125, 255, cv.THRESH_BINARY_INV)
#     thresh = cv.Canny(image,50,100,apertureSize = 3)
#     thresh = cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
    cv.imshow("org", thresh)
    cv.waitKey(0)

    
    contours, hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#     cnts = sorted(contours, key=lambda x: cv.contourArea(x), reverse=True)[:1]
#     blank = np.zeros(image.shape, np.uint8)
    c = max(contours, key = cv.contourArea)
    x,y,w,h = cv.boundingRect(c)    
    image = image[y:y+h+1, x:x+w+1]
    cv.imshow("cn", image)
    cv.waitKey(0)
    
    edges = cv.Canny(image,40,80)
#     ret, edges = cv.threshold(image, 115, 255, cv.THRESH_BINARY_INV)
    cv.imshow("org", edges)
    cv.waitKey(0)

    lines = cv.HoughLines(edges,1,np.pi/180,200)
    
    points = np.array([[line[0][0], line[0][1]] for line in lines])
    pi_val = math.pi
    
    v1 = list(filter(lambda x: x[1]>=0 and x[1]<pi_val/4, points))
    v2 = list(filter(lambda x: x[1]>=(3*pi_val)/4 and x[1]<(5*pi_val)/4, points))
    v3 = list(filter(lambda x: x[1]>=(7*pi_val)/4 and x[1]<=pi_val*2, points))
    
    vertical = v1 + v2 + v3
    
    h1 = list(filter(lambda x: x[1]>=pi_val/4 and x[1]<(3*pi_val)/4, points))
    h2 = list(filter(lambda x: x[1]>=(5*pi_val)/4 and x[1]<(7*pi_val)/4, points))
    
    horizontal = h1 + h2
    
    h_lines = my_cluster(horizontal, 15)
    v_lines = my_cluster(vertical, 15)
    
    lines = h_lines + v_lines
    
    for line in lines:
        print(line)
        
    line_image = image.copy()
    
    for rho, theta in lines:
#         rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        
        cv.line(line_image,(x1,y1),(x2,y2),(0,0,255),2)

    cv.namedWindow('lines',cv.WINDOW_NORMAL)
    cv.resizeWindow('lines', 600,600)
    cv.imshow("lines", line_image)
    cv.waitKey(0)

    
#     for i in range(len(h_lines)-1):
#         for j in range(len(v_lines)-1):
#             hl1, hl2 = h_lines[i], h_lines[i+1]
#             vl1, vl2 = v_lines[j], v_lines[j+1]
            
#             p1 = intersection_bw_2_lines(hl1, vl1)
#             p2 = intersection_bw_2_lines(hl1, vl2)
#             p3 = intersection_bw_2_lines(hl2, vl1)
#             p4 = intersection_bw_2_lines(hl2, vl2)
            
#             cell = image[p1[1]:p3[1]+1, p1[0]:p2[0]+1]
            
#             cv.imshow("cell", cell)
#             cv.waitKey(0)

    p1 = intersection_bw_2_lines(h_lines[0], v_lines[0])
    p2 = intersection_bw_2_lines(h_lines[0], v_lines[-1])
    p3 = intersection_bw_2_lines(h_lines[-1], v_lines[0])
    p4 = intersection_bw_2_lines(h_lines[-1], v_lines[-1])

    pts1 = np.float32([p1, p2, p3, p4])
    w,h = image.shape
    pts2 = np.float32([[0,0],[h,0],[0,w],[h,w]])
    M = cv.getPerspectiveTransform(pts1,pts2)
    dst = cv.warpPerspective(image,M,(h,w))
    
    cv.imshow("final", dst)
    cv.waitKey(0)

    cv.destroyAllWindows()

[26.         1.5184364]
[77.         1.5009831]
[121.          1.5009831]
[166.          1.5009831]
[207.          1.5184364]
[253.          1.5184364]
[297.          1.5184364]
[342.          1.5184364]
[386.          1.5184364]
[431.          1.5184364]
[476.          1.5184364]
[522.          1.5184364]
[568.          1.5184364]
[615.          1.5184364]
[662.          1.5184364]
[716.          1.5358897]
[759.          1.5358897]
[781.          1.5184364]
[821.          1.5184364]
[-5.         3.0892327]
[-69.          3.0892327]
[-132.           3.0892327]
[-194.           3.0892327]
[-257.           3.0892327]
[-320.           3.0892327]
[-384.           3.0892327]
[-473.           3.0892327]


In [4]:
dst_edges = cv.Canny(dst,40,80)

lines = cv.HoughLines(dst_edges,1,np.pi/180,200)

line_image = dst.copy()
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv.line(line_image,(x1,y1),(x2,y2),(0,0,255),2)

cv.namedWindow('lines',cv.WINDOW_NORMAL)
cv.resizeWindow('lines', 600,600)
cv.imshow("lines", line_image)
cv.waitKey(0)

cv.destroyAllWindows()

In [42]:
import imutils

image = cv.imread(path+"sample1.jpg", 0)
edges = cv.Canny(image, 40, 90)
edges = cv.dilate(edges, None, iterations=1)
# cv.namedWindow('canny',cv.WINDOW_NORMAL)
# cv.resizeWindow('canny', 600,600)
cv.imshow("canny", edges)
cv.waitKey(0)

cnts = cv.findContours(edges.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = max(cnts, key=cv.contourArea)

rect = cv.minAreaRect(c)
box = cv.boxPoints(rect)
box = np.int0(box)
im = cv.drawContours(image,[box],0,(0,0,255),2)

# extLeft = tuple(c[c[:, :, 0].argmin()][0])
# extRight = tuple(c[c[:, :, 0].argmax()][0])
# extTop = tuple(c[c[:, :, 1].argmin()][0])
# extBot = tuple(c[c[:, :, 1].argmax()][0])

# cv.drawContours(image, [c], -1, (0, 255, 255), 2)
# cv.circle(image, extLeft, 8, (0, 0, 255), -1)
# cv.circle(image, extRight, 8, (0, 255, 0), -1)
# cv.circle(image, extTop, 8, (255, 0, 0), -1)
# cv.circle(image, extBot, 8, (255, 255, 0), -1)

print(box, abs(box))

cv.namedWindow('image',cv.WINDOW_NORMAL)
cv.resizeWindow('image', 600,600)
cv.imshow("image", im)
cv.waitKey(0)
        
box[0][0], box[1][0], box[2][0], box[-1][0] = box[0][0]+20, box[1][0]+20, box[2][0]-20, box[-1][0]-20 
box[box < 0] = 0

w,h = 600, 450
pts1 = np.float32(box)
# w,h = image.shape
pts2 = np.float32([[0,w], [0,0], [h,0], [h,w]])
M = cv.getPerspectiveTransform(pts1,pts2)
dst = cv.warpPerspective(image,M,(h,w))

cv.imshow("final", dst)
cv.waitKey(0)

cv.destroyAllWindows()

[[  0 852]
 [-35 160]
 [512 132]
 [548 824]] [[  0 852]
 [ 35 160]
 [512 132]
 [548 824]]
